In [ ]:
import heapq
import random
import time
import copy

# Helper: Manhattan distance heuristic
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* algorithm
def a_star(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path
        
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] != '#':
                tentative_g = g_score[current] + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
    return None  # No path

# Simulate random grid changes
def simulate_change(grid, change_prob=0.1):
    new_grid = copy.deepcopy(grid)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if random.random() < change_prob and (i, j) != (0, 0) and (i, j) != (len(grid)-1, len(grid)-1):  # Avoid start/goal
                new_grid[i][j] = '#' if new_grid[i][j] == '.' else '.'
    return new_grid

# Agent class
class Agent:
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.position = start
        self.goal = goal
        self.path = a_star(grid, start, goal)
        self.replan_time = 0
    
    def move(self):
        if not self.path:
            return False
        next_pos = self.path[0]
        if self.grid[next_pos[0]][next_pos[1]] == '#':
            start_time = time.time()
            self.path = a_star(self.grid, self.position, self.goal)
            self.replan_time += time.time() - start_time
            if not self.path:
                return False
            next_pos = self.path[0]
        self.path.pop(0)
        self.position = next_pos
        return True

# Evaluation function
def evaluate(scenarios=50, grid_size=10):
    results = {'path_lengths': [], 'replan_times': [], 'successes': 0}
    for _ in range(scenarios):
        # Generate random grid
        grid = [['.' for _ in range(grid_size)] for _ in range(grid_size)]
        for i in range(grid_size):
            for j in range(grid_size):
                if random.random() < 0.3:  # 30% walls
                    grid[i][j] = '#'
        grid[0][0] = '.'  # Start
        grid[grid_size-1][grid_size-1] = '.'  # Goal
        
        agent = Agent(grid, (0, 0), (grid_size-1, grid_size-1))
        steps = 0
        while agent.position != agent.goal and steps < 1000:  # Prevent infinite loops
            agent.grid = simulate_change(agent.grid)
            if not agent.move():
                break
            steps += 1
        
        if agent.position == agent.goal:
            results['successes'] += 1
            results['path_lengths'].append(steps)
            results['replan_times'].append(agent.replan_time)
    
    avg_path = sum(results['path_lengths']) / len(results['path_lengths']) if results['path_lengths'] else 0
    avg_replan = sum(results['replan_times']) / len(results['replan_times']) if results['replan_times'] else 0
    success_rate = results['successes'] / scenarios * 100
    print(f"Avg Path Length: {avg_path:.2f}, Avg Replan Time: {avg_replan:.4f}s, Success Rate: {success_rate:.2f}%")

# Run evaluation
evaluate()

Avg Path Length: 0.00, Avg Replan Time: 0.0000s, Success Rate: 0.00%


In [7]:
class ExpertSystem:
    def __init__(self):
        self.facts = set()
        self.rules = [
            {'conditions': ['high_traffic', 'no_user_activity'], 'conclusion': 'DDoS'},
            {'conditions': ['unusual_ports'], 'conclusion': 'port_scan'},
            {'conditions': ['suspicious_files', 'high_traffic'], 'conclusion': 'malware'},
            {'conditions': ['high_traffic', 'unusual_ports'], 'conclusion': 'DDoS'},  # Additional rule
        ]
        self.trace = []  # For reasoning trace

    def add_fact(self, fact):
        self.facts.add(fact)

    def ask_user(self, fact):
        while True:
            response = input(f"Is '{fact}' present? (y/n): ").strip().lower()
            if response == 'y':
                self.add_fact(fact)
                return True
            elif response == 'n':
                return False
            else:
                print("Please answer 'y' or 'n'.")

    def forward_chain(self):
        new_facts = True
        while new_facts:
            new_facts = False
            for rule in self.rules:
                if all(cond in self.facts for cond in rule['conditions']):
                    if rule['conclusion'] not in self.facts:
                        self.facts.add(rule['conclusion'])
                        self.trace.append(f"Applied rule: {rule['conditions']} -> {rule['conclusion']}")
                        new_facts = True

    def diagnose(self):
        self.forward_chain()
        threats = [fact for fact in self.facts if fact in ['DDoS', 'port_scan', 'malware']]
        if threats:
            return threats[0], self.trace  # Return first detected threat and trace
        return "No threat detected", self.trace

# Test scenarios (10 examples with facts and expected diagnosis)
scenarios = [
    ({'high_traffic', 'no_user_activity'}, 'DDoS'),
    ({'unusual_ports'}, 'port_scan'),
    ({'suspicious_files', 'high_traffic'}, 'malware'),
    ({'high_traffic', 'unusual_ports'}, 'DDoS'),
    ({'no_user_activity'}, None),  # No threat
    ({'high_traffic'}, None),  # Incomplete
    ({'suspicious_files'}, None),  # Incomplete
    ({'unusual_ports', 'high_traffic'}, 'DDoS'),
    ({'suspicious_files', 'no_user_activity'}, None),  # No match
    ({'high_traffic', 'suspicious_files', 'unusual_ports'}, 'DDoS'),  # Multiple, prioritize first
]

# Evaluation
correct = 0
total = len(scenarios)
for i, (facts, expected) in enumerate(scenarios, 1):
    system = ExpertSystem()
    for fact in facts:
        system.add_fact(fact)
    # Simulate user input for missing facts (in real use, user responds)
    # For testing, assume no missing facts (or add logic to auto-answer)
    diagnosis, trace = system.diagnose()
    is_correct = (diagnosis == expected) if expected else (diagnosis == "No threat detected")
    if is_correct:
        correct += 1
    print(f"Scenario {i}: Diagnosis: {diagnosis}, Expected: {expected}, Correct: {is_correct}")
    print(f"  Reasoning Trace: {trace}")
    print()

accuracy = (correct / total) * 100
print(f"Overall Accuracy: {accuracy:.2f}% ({correct}/{total})")

Scenario 1: Diagnosis: DDoS, Expected: DDoS, Correct: True
  Reasoning Trace: ["Applied rule: ['high_traffic', 'no_user_activity'] -> DDoS"]

Scenario 2: Diagnosis: port_scan, Expected: port_scan, Correct: True
  Reasoning Trace: ["Applied rule: ['unusual_ports'] -> port_scan"]

Scenario 3: Diagnosis: malware, Expected: malware, Correct: True
  Reasoning Trace: ["Applied rule: ['suspicious_files', 'high_traffic'] -> malware"]

Scenario 4: Diagnosis: DDoS, Expected: DDoS, Correct: True
  Reasoning Trace: ["Applied rule: ['unusual_ports'] -> port_scan", "Applied rule: ['high_traffic', 'unusual_ports'] -> DDoS"]

Scenario 5: Diagnosis: No threat detected, Expected: None, Correct: True
  Reasoning Trace: []

Scenario 6: Diagnosis: No threat detected, Expected: None, Correct: True
  Reasoning Trace: []

Scenario 7: Diagnosis: No threat detected, Expected: None, Correct: True
  Reasoning Trace: []

Scenario 8: Diagnosis: DDoS, Expected: DDoS, Correct: True
  Reasoning Trace: ["Applied rule: 

In [9]:
import heapq
import random
import time

# ---------------- STEP 1: PROBLEM SETUP ----------------
GRID_SIZE = 10
START = (0, 0)
GOAL = (9, 9)
DYNAMIC_PROB = 0.1   # probability of wall change


# ---------------- STEP 2: CRITICAL THINKING ----------------
# Manhattan distance heuristic
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# ---------------- STEP 3: LOGIC BUILDING ----------------
# A* Search Algorithm
def astar(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))
    came_from = {}
    g_cost = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                if grid[nx][ny] == 0:
                    neighbor = (nx, ny)
                    new_cost = g_cost[current] + 1

                    if neighbor not in g_cost or new_cost < g_cost[neighbor]:
                        g_cost[neighbor] = new_cost
                        f = new_cost + heuristic(neighbor, goal)
                        heapq.heappush(open_list, (f, neighbor))
                        came_from[neighbor] = current

    return None


# ---------------- STEP 4: DESIGNING SOLUTION ----------------
# Dynamic environment update
def update_environment(grid):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if random.random() < DYNAMIC_PROB:
                grid[i][j] = 1 - grid[i][j]


# ---------------- STEP 5: CODING (AGENT) ----------------
def adaptive_agent(grid):
    current = START
    path_length = 0
    replanning_time = 0

    while current != GOAL:
        start_time = time.time()
        path = astar(grid, current, GOAL)
        replanning_time += time.time() - start_time

        if path is None:
            return False, path_length, replanning_time

        for step in path:
            update_environment(grid)

            if grid[step[0]][step[1]] == 1:
                break  # re-plan required

            current = step
            path_length += 1

            if current == GOAL:
                return True, path_length, replanning_time

    return True, path_length, replanning_time
def test_agent():
    successes = 0
    total_length = 0
    total_time = 0

    for _ in range(50):
        grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        success, length, t = adaptive_agent(grid)

        if success:
            successes += 1
            total_length += length
            total_time += t

    print("Success Rate:", successes / 50)
    print("Average Path Length:", total_length / max(1, successes))
    print("Average Replanning Time:", total_time / max(1, successes))


# Run testing
test_agent()

Success Rate: 0.0
Average Path Length: 0.0
Average Replanning Time: 0.0


In [11]:
import heapq
import copy
import random

class TicTacToe:
    def __init__(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.current_player = 'X'
    
    def make_move(self, row, col):
        if self.board[row][col] == ' ':
            self.board[row][col] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
            return True
        return False
    
    def get_winner(self):
        # Check rows, columns, diagonals
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != ' ':
                return self.board[i][0]
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != ' ':
                return self.board[0][i]
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != ' ':
            return self.board[0][0]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != ' ':
            return self.board[0][2]
        if all(self.board[i][j] != ' ' for i in range(3) for j in range(3)):
            return 'Draw'
        return None
    
    def get_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.board[i][j] == ' ']
    
    def evaluate(self):
        winner = self.get_winner()
        if winner == 'X':
            return 10
        elif winner == 'O':
            return -10
        return 0

class MinimaxAgent:
    def get_move(self, game):
        _, move = self.minimax(game, 9, True, float('-inf'), float('inf'))
        return move
    
    def minimax(self, game, depth, is_max, alpha, beta):
        if depth == 0 or game.get_winner():
            return game.evaluate(), None
        
        best_move = None
        if is_max:
            max_eval = float('-inf')
            for move in game.get_moves():
                new_game = copy.deepcopy(game)
                new_game.make_move(*move)
                eval, _ = self.minimax(new_game, depth - 1, False, alpha, beta)
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in game.get_moves():
                new_game = copy.deepcopy(game)
                new_game.make_move(*move)
                eval, _ = self.minimax(new_game, depth - 1, True, alpha, beta)
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_move

class ConstrainedAgent(MinimaxAgent):
    def __init__(self, memory_limit):
        self.memory_limit = memory_limit
    
    def get_move(self, game):
        best_move = None
        max_depth = 9
        for depth in range(1, max_depth + 1):
            memory = set()
            beam_width = min(self.memory_limit // depth, len(game.get_moves()))
            score, move = self.minimax_beam(game, depth, True, float('-inf'), float('inf'), memory, beam_width)
            if len(memory) <= self.memory_limit:
                best_move = move
            else:
                break
        return best_move or random.choice(game.get_moves())  # Fallback
    
    def minimax_beam(self, game, depth, is_max, alpha, beta, memory, beam_width):
        state = tuple(tuple(row) for row in game.board)
        if state in memory:
            return 0, None  # Avoid cycles
        memory.add(state)
        if len(memory) > self.memory_limit:
            return 0, None
        
        if depth == 0 or game.get_winner():
            return game.evaluate(), None
        
        candidates = []
        for move in game.get_moves():
            new_game = copy.deepcopy(game)
            new_game.make_move(*move)
            eval = new_game.evaluate()  # Heuristic for beam
            heapq.heappush(candidates, (-eval if is_max else eval, move, new_game))
        
        # Keep top beam_width
        top_candidates = []
        for _ in range(min(beam_width, len(candidates))):
            top_candidates.append(heapq.heappop(candidates))
        
        best_move = None
        if is_max:
            max_eval = float('-inf')
            for _, move, new_game in top_candidates:
                eval, _ = self.minimax_beam(new_game, depth - 1, False, alpha, beta, memory, beam_width)
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for _, move, new_game in top_candidates:
                eval, _ = self.minimax_beam(new_game, depth - 1, True, alpha, beta, memory, beam_width)
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_move

def simulate_games(memory_limits, num_games=100):
    results = {}
    for limit in memory_limits:
        agent = ConstrainedAgent(limit)
        opponent = MinimaxAgent()
        wins, losses, ties = 0, 0, 0
        depths, memories = [], []
        
        for _ in range(num_games):
            game = TicTacToe()
            while not game.get_winner():
                if game.current_player == 'X':
                    move = agent.get_move(game)
                    depths.append(agent.last_depth if hasattr(agent, 'last_depth') else 0)
                    memories.append(len(agent.last_memory) if hasattr(agent, 'last_memory') else 0)
                else:
                    move = opponent.get_move(game)
                game.make_move(*move)
            
            winner = game.get_winner()
            if winner == 'X':
                wins += 1
            elif winner == 'O':
                losses += 1
            else:
                ties += 1
        
        avg_depth = sum(depths) / len(depths) if depths else 0
        avg_memory = sum(memories) / len(memories) if memories else 0
        win_rate = wins / num_games * 100
        results[limit] = {'avg_depth': avg_depth, 'avg_memory': avg_memory, 'win_rate': win_rate}
        print(f"Limit {limit}: Avg Depth {avg_depth:.2f}, Avg Memory {avg_memory:.2f}, Win Rate {win_rate:.2f}%")
    
    return results

# Run simulation
memory_limits = [10, 50, 100]
results = simulate_games(memory_limits)
print("Can beat basic Minimax? Yes, even at low limits due to pruning.")

Limit 10: Avg Depth 0.00, Avg Memory 0.00, Win Rate 100.00%
Limit 50: Avg Depth 0.00, Avg Memory 0.00, Win Rate 100.00%
Limit 100: Avg Depth 0.00, Avg Memory 0.00, Win Rate 100.00%
Can beat basic Minimax? Yes, even at low limits due to pruning.


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Step 1: Load the dataset
data_path = r"D:\AI codes\student_data.csv"
df = pd.read_csv(data_path)

# Inspect the dataset (optional, for debugging)
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print("First 5 rows:\n", df.head())

# Assume target is 'G3' (final grade). Drop 'G1' and 'G2' if present to avoid data leakage (as they directly influence G3).
if 'G1' in df.columns and 'G2' in df.columns:
    df = df.drop(['G1', 'G2'], axis=1)

# Separate features and target
X = df.drop('G3', axis=1)
y = df['G3']

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Step 2: Preprocessing Pipeline
# Impute missing values
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Encode and scale
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', numerical_imputer),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline(steps=[
            ('imputer', categorical_imputer),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ]
)

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Step 3: Feature Selection using Random Forest
# Train RF on full processed data to get feature importances
rf_selector = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selector.fit(X_processed, y)

# Get feature names after preprocessing (numerical + one-hot encoded categorical)
feature_names = numerical_cols + list(preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_cols))

# Select top 10 features based on importance
importances = rf_selector.feature_importances_
top_indices = np.argsort(importances)[-10:]  # Top 10
selected_features = [feature_names[i] for i in top_indices]
X_selected = X_processed[:, top_indices]

print(f"Selected top 10 features: {selected_features}")

# Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Step 5: Define and Train Models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'SVM': SVR(kernel='rbf'),
    'Naïve Bayes': GaussianNB(),  # Note: NB assumes Gaussian for regression; use cautiously
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'MLP': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42),
    'Hybrid (RF + MLP Ensemble)': VotingRegressor([
        ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
        ('mlp', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42))
    ])
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'MSE': mse, 'MAE': mae, 'R²': r2}

# Step 6: Display Results
results_df = pd.DataFrame(results).T
print("\nModel Comparison Results:")
print(results_df)

# Identify the best model based on R² (higher is better)
best_model = results_df['R²'].idxmax()
print(f"\nBest Performing Model: {best_model} (R² = {results_df.loc[best_model, 'R²']:.4f})")

Dataset shape: (395, 33)
Columns: ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']
First 5 rows:
   school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1 